# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 09**

**Fecha**: 8/04/2025

**Nombre del Estudiante**:
- Alberto Renteria Camacho
- Ximena Isaac Horta
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto

**Profesor**: Pablo Camarillo Ramirez

In [49]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [50]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka-Watermarking") \
    .master("spark://3e150af84880:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [51]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "b2513693f9e5:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [52]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

In [53]:
from pyspark.sql.functions import explode, split, length

words = kafka_df.select(explode(split(kafka_df.value, " ")).alias("word"), "timestamp")
words = words.withColumn("lenght", length(words.word))
words.printSchema()

root
 |-- word: string (nullable = false)
 |-- timestamp: timestamp (nullable = true)
 |-- lenght: integer (nullable = false)



### Opción 1

In [46]:
from pyspark.sql.functions import window, avg, sum, max, min, count
windowed_counts =  words \
                        .withWatermark("timestamp", "1 minutes") \
                        .groupBy(window(words.timestamp, 
                                        "40 seconds", # Window duration 
                                        "20 seconds"), # Slide duration
                            words.word) \
                        .agg(avg("lenght").alias("Average"), \
                            sum("lenght").alias("Total"), \
                            min("lenght").alias("Min"), \
                            max("lenght").alias("Max"),
                            count("word").alias("Count")) 

### Opción 2

In [54]:
from pyspark.sql.functions import window, avg, sum, max, min, count
windowed_counts =  words \
                        .withWatermark("timestamp", "1 minutes") \
                        .groupBy(window(words.timestamp, 
                                        "40 seconds", # Window duration 
                                        "20 seconds"), # Slide duration
                            ) \
                        .agg(avg("lenght").alias("Average"), \
                            sum("lenght").alias("Total"), \
                            min("lenght").alias("Min"), \
                            max("lenght").alias("Max")) 

### Configuración del "Sink" del stream

In [47]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

query = windowed_counts \
                .writeStream \
                .outputMode("update") \
                .trigger(processingTime='30 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(100)

25/04/10 00:37:52 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0ead325c-e094-48ba-8aa4-ef000df6bca1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/10 00:37:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/10 00:37:52 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-------+-----+---+---+-----+
|window|word|Average|Total|Min|Max|Count|
+------+----+-------+-----+---+---+-----+
+------+----+-------+-----+---+---+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+--------+-------+-----+---+---+-----+
|window                                    |word    |Average|Total|Min|Max|Count|
+------------------------------------------+--------+-------+-----+---+---+-----+
|{2025-04-10 00:37:40, 2025-04-10 00:38:20}|palabras|8.0    |16   |8  |8  |2    |
|{2025-04-10 00:37:20, 2025-04-10 00:38:00}|palabras|8.0    |16   |8  |8  |2    |
+------------------------------------------+--------+-------+-----+---+---+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+--------+-------+-----+---+---+-----+
|window                                    |word    |Average|Total|Min|Max|Count|
+------------------------------------------+--------+-------+-----+---+---+-----+
|{2025-04-10 00:38:00, 2025-04-10 00:38:40}|palabras|8.0    |8    |8  |8  |1    |
|{2025-04-10 00:37:40, 2025-04-10 00:38:20}|palabras|8.0    |24   |8  |8  |3    |
|{2025-04-10 00:38:00, 2025-04-10 00:38:40}|apra    |4.0    |8    |4  |4  |2    |
|{2025-04-10 00:37:40, 2025-04-10 00:38:20}|lo      |2.0    |2    |2  |2  |1    |
|{2025-04-10 00:38:00, 2025-04-10 00:38:40}|hay     |3.0    |3    |3  |3  |1    |
|{2025-04-10 00:38:00, 2025-04-10 00:38:40}|en      |2.0    |2    |2  |2  |1    |
|{2025-04-10 00:37:40, 2025-04-10 00:38:20}|en      |2.0    |2    |2  |2  |1    |
|{2025-04-10 00:38:00, 2025-04-10 00:38:40}|para    |4.0    |4    |4  |4  |1    |
|

-------------------------------------------
Batch: 3
-------------------------------------------
+------+----+-------+-----+---+---+-----+
|window|word|Average|Total|Min|Max|Count|
+------+----+-------+-----+---+---+-----+
+------+----+-------+-----+---+---+-----+



False

In [55]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

query = windowed_counts \
                .writeStream \
                .outputMode("update") \
                .trigger(processingTime='30 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(60)

25/04/10 00:40:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f7156eb3-ec24-47bf-8b48-d3a17356fb58. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/10 00:40:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/10 00:40:18 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+---+---+
|window|Average|Total|Min|Max|
+------+-------+-----+---+---+
+------+-------+-----+---+---+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----+---+---+
|window                                    |Average|Total|Min|Max|
+------------------------------------------+-------+-----+---+---+
|{2025-04-10 00:40:20, 2025-04-10 00:41:00}|6.5    |39   |6  |7  |
|{2025-04-10 00:40:00, 2025-04-10 00:40:40}|6.5    |39   |6  |7  |
+------------------------------------------+-------+-----+---+---+



False

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------------------+-----+---+---+
|window                                    |Average           |Total|Min|Max|
+------------------------------------------+------------------+-----+---+---+
|{2025-04-10 00:40:40, 2025-04-10 00:41:20}|3.4285714285714284|24   |1  |7  |
|{2025-04-10 00:41:20, 2025-04-10 00:42:00}|22.0              |22   |22 |22 |
|{2025-04-10 00:41:00, 2025-04-10 00:41:40}|5.75              |46   |1  |22 |
+------------------------------------------+------------------+-----+---+---+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-------+-----+---+---+
|window|Average|Total|Min|Max|
+------+-------+-----+---+---+
+------+-------+-----+---+---+



In [48]:
query.stop()
sc.stop()